To run this code, simply start running it sequentially. In the sixth code block, you can modify parameters such as the poem starter word and the poem rhyme scheme. Feel free to experiment with those variables.

It is important to note that once you've run the code for the first time, in additional runs you only have to run from the sixth code block and onwards to generate more poems.

In [1]:
import tensorflow as tf
import keras
import numpy as np
from keras.preprocessing.text import Tokenizer
import pronouncing
import gensim.downloader
import pickle
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

2022-12-10 16:38:32.649084: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-10 16:38:32.724632: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-10 16:38:33.113626: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/

In [2]:
class Model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [3]:
with open('tokenizer', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)


vocab_size = len(tokenizer.word_index) + 1
# Embedding size
embedding_dim = 512
# RNN size
rnn_units = 1024
    
model = Model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.load_weights('model_poems')

2022-12-10 16:39:08.920282: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:39:08.923637: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:39:08.923737: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-10 16:39:08.924070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [4]:
class OneStep(tf.keras.Model):
  def __init__(self, model, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model

  def generate_one_step(self, inputs, states=None):
    
    temp = tokenizer.texts_to_sequences(inputs)
    if len(temp[0]) == 0:
      temp = [[1]]
    input_ids = np.array(temp)
    predicted_logits, states = self.model(input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = tokenizer.sequences_to_texts([predicted_ids.numpy()])
    
    return predicted_chars, states

In [5]:
def get_rhyme_with_max_embedding(base_word, context_words):
  
  all_rhyme_words = pronouncing.rhymes(base_word)

  best_count = float('-inf')
  best_word = ""
  for r in all_rhyme_words:
    count = 0
    seen_content = 1
    for c in context_words:
      if r in glove_vectors and c[0] in glove_vectors:
        count += glove_vectors.similarity(r, c[0])
        seen_content += 1
    
    count = count / seen_content
    
    
    if count >= best_count:
      best_word = r
      best_count = count

  return best_word

one_step_model = OneStep(model)

In [18]:
## CONFIGURE STARTER WORD AND RHYME SCHEME HERE
# RUN ALL CELLS HERE AND BELOW TO SEE NEW POEMS

next_char = ['twas']
rhyme_scheme = 'ABABCCDD'

In [19]:
states = None
result = [next_char]

breaks = rhyme_scheme.count(" ")
lines = len(rhyme_scheme) - breaks
n = 0
prev_char = []


while n < lines:
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  
  if next_char == ['\n']:
    if prev_char != ['\n']:
      result.append(next_char)
      n += 1
  else:
    result.append(next_char)
    
  prev_char = next_char

In [20]:
line_count = 0
rhymes = {}

temp_scheme = rhyme_scheme.replace(" ", "")
i = 0
start_index = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    rhyme_index = temp_scheme[line_count]
    if rhyme_index in rhymes:
      base_rhyme_word = rhymes[rhyme_index]
      embedding_words = result[start_index:i-1]
      rhymed_word = get_rhyme_with_max_embedding(base_rhyme_word[0], embedding_words)
      result[i-1] = [rhymed_word]
      rhymes[rhyme_index] = result[i-1]
      
    else:
      rhymes[rhyme_index] = result[i-1]
    
    start_index = i + 1
    line_count += 1
    
    
start_index = 0
line_count = 0
for i in range(len(result)):
  if result[i] == ['\n']:
    if rhyme_scheme[line_count] == " ":
        result[start_index - 1] = ['\n\n']
        
    line_count += 1
    start_index = i + 1

In [21]:
poem = " "
for l in result:
  poem += l[0]
  poem += " "
print(poem)

 twas shaken with a little boy 
 on christmas eve a long time 
 on happy night when she slept and enjoy 
 while each year on happy and have crime 
 our grateful and grim that then hand 
 and in the and 
 the berries on the holly bright 
 and as the ancient voice before the might 
 
